In [18]:
import matplotlib.pyplot as plt
import numpy as np
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys

In [32]:
interact_manual(plotGraph, mass_1_x=widgets.FloatSlider(min=-50, max=50, value=0), mass_1_y=widgets.FloatSlider(min=-50, max=50, value=0), mass_2_x=widgets.FloatSlider(min=-50, max=50, value=0.2), mass_2_y=widgets.FloatSlider(min=-50, max=50, value=0), alpha=widgets.FloatSlider(min=0, max=50, value=1), moving_mass_x=widgets.FloatSlider(min=-50, max=50, value=1), moving_mass_y=widgets.FloatSlider(min=-50, max=50, value=0), moving_mass_x_prime=widgets.FloatSlider(min=-50, max=50, value=0), moving_mass_y_prime=widgets.FloatSlider(min=-50, max=50, value=2*np.pi), its=widgets.IntSlider(min=1, max=10000, value=400), delta=widgets.FloatSlider(min=0, max=1.0, value=0.005, readout_format=".6f", step=0.000001))


def plotGraph(mass_1_x, mass_1_y, mass_2_x, mass_2_y, alpha, moving_mass_x, moving_mass_y, moving_mass_x_prime, moving_mass_y_prime, its, delta):
    print(
        f"original: m1: ({mass_1_x}, {mass_1_y}), m2: ({mass_2_x}, {mass_2_y}), m: ({moving_mass_x}, {moving_mass_y})")

    # Reposition the masses such that M1 is at the origin and M2 lies along the y-axis
    # First, move all points
    # Then rotate

    mass_2_x -= mass_1_x
    mass_2_y -= mass_1_y
    moving_mass_x -= mass_1_x
    moving_mass_y -= mass_1_y

    m_theta = np.arctan2(moving_mass_y, moving_mass_x)
    m2_theta = np.arctan2(mass_2_y, mass_2_x)

    mass_2_x = np.sqrt(mass_2_x ** 2 + mass_2_y ** 2)
    mass_2_y = 0
    mass_1_x = 0
    mass_1_y = 0

    # Subtract m2_theta from current m_theta

    m_theta -= m2_theta
    m_r = np.sqrt(moving_mass_x ** 2 + moving_mass_y ** 2)

    # Convert back to cartesian coordinates

    moving_mass_x = m_r * np.cos(m_theta)
    moving_mass_y = m_r * np.sin(m_theta)

    # m1 and m2 should now lay along the x-axis and m should be rotated relative to them
    print(
        f"final: m1: ({mass_1_x}, {mass_1_y}), m2: ({mass_2_x}, {mass_2_y}), m: ({moving_mass_x}, {moving_mass_y})")

    # declare arrays that will be appended to in loop
    x = [moving_mass_x]
    y = [moving_mass_y]
    x_prime = [moving_mass_x_prime]
    y_prime = [moving_mass_y_prime]

    # We have the following equations of motion
    # x ̇_(n+1)=x ̇_n-δt4π^2 (x_(n+1)/〖r_1^3〗_(n+1) +α(x_(n+1)-d)/〖r_2^3〗_(n+1) )
    # y ̇_(n+1)=y ̇_n-δt4π^2 y_(n+1) (1/〖r_1^3〗_(n+1) +α/〖r_2^3〗_(n+1) )

    for n in range(its):
        # calculate next x, y, x', and y'
        x_n_plus_one = x[n] + delta * x_prime[n]
        x.append(x_n_plus_one)
        y_n_plus_one = y[n] + delta * y_prime[n]
        y.append(y_n_plus_one)

        x_prime_n_plus_one = x_prime[n] - delta * 4 * (np.pi ** 2) * (x[n + 1] / np.power((x[n + 1] ** 2) + (
            y[n + 1] ** 2), 3 / 2) + alpha * (x[n+1] - mass_2_x) / np.power((x[n + 1] - mass_2_x) ** 2 + y[n + 1] ** 2, 3 / 2))
        x_prime.append(x_prime_n_plus_one)

        y_prime_n_plus_one = y_prime[n] - delta * 4 * (np.pi ** 2) * y[n + 1] * (1 / np.power(
            (x[n + 1] ** 2) + (y[n + 1] ** 2), 3 / 2) + 1 / np.power((x[n + 1] - mass_2_x) ** 2 + y[n + 1] ** 2, 3 / 2))
        y_prime.append(y_prime_n_plus_one)

    plt.plot(x, y)

    masses = [[mass_1_x, mass_2_x, moving_mass_x],
              [mass_1_y, mass_2_y, moving_mass_y]]
    colors = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255]])

    plt.scatter(masses[0], masses[1], c=colors / 255)

    labels = ["m1", "m2", "m"]
    for i, txt in enumerate(labels):
        plt.annotate(txt, (masses[0][i], masses[1][i]))

    plt.show()


interactive(children=(FloatSlider(value=0.0, description='mass_1_x', max=50.0, min=-50.0), FloatSlider(value=0…